In [1]:
import torch
import ttnn

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening device driver
2023-10-25 17:28:57.421 | INFO     | SiliconDriver   - Detected 1 PCI device
2023-10-25 17:28:57.431 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-10-25 17:28:57.431 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2023-10-25 17:28:57.431 | INFO     | SiliconDriver   - Using 1 Hugepages/NumHostMemChannels for TTDevice (pci_interface_id: 0 device_id: 0xfaca revision: 0)
2023-10-25 17:28:57.433 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaN

In [2]:
import torch
import ttnn

# Use tensors of rank 4
Notice that the dimensions for the height and width are multiples of 32

In [3]:
n = 1
c = 1
h = 31
w = 32 * 2

# Initialize tensors a and b with random values using torch

In [4]:
torch_a = torch.rand((n, c, h, w), dtype=torch.bfloat16)
torch_b = torch.rand((n, c, h, w), dtype=torch.bfloat16)
a = ttnn.from_torch(torch_a)
b = ttnn.from_torch(torch_b)


# Add tensor a and b

In [5]:
output = a + b

# Print the result of the add

In [6]:
print("Printing ttnn tensor")
print(output.shape)
print(output[0, 0, :1])

Printing ttnn tensor
[1, 1, 31, 64]
Tensor([ [1.1875, 1.02344, 0.898438, 1.07812, 0.488281, 1.25781, 0.457031, 1.04688, 0.777344, 0.652344, 0.757812, 1.46875, 1.74219, 0.433594, 1.54688, 0.726562, 0.929688, 0.449219, 0.6875, 0.40625, 0.878906, 1.03125, 1.14844, 0.695312, 1.63281, 1.01562, 0.769531, 1.27344, 0.976562, 1.24219, 0.492188, 0.734375, 1.20312, 1.05469, 1.07812, 1.84375, 0.753906, 0.113281, 1.0625, 1.60156, 0.847656, 0.421875, 1.14844, 1.16406, 1.05469, 1.10938, 0.773438, 0.882812, 1.19531, 1.125, 0.3125, 0.53125, 1.35156, 1.07031, 0.457031, 1.08594, 1.15625, 1.72656, 0.875, 0.386719, 1.39844, 1.45312, 1.07031, 1.19531]], dtype=bfloat16 )



# Free tensor

In [7]:
del output

# Free the device

In [8]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
